In [1]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
import os
import re

Using TensorFlow backend.


In [2]:
lines= pd.read_table('/home/anusha/pCloudDrive/thesis/translator/Word-Level-Eng-Mar-NMT/Telugu-Seq2Seq-stemmer/data/allsimplew5.txt'
, names=['eng', 'tel'])
linestest=pd.read_table('/home/anusha/pCloudDrive/thesis/translator/Word-Level-Eng-Mar-NMT/Telugu-Seq2Seq-stemmer/data/allsimpletest2.txt', names=['eng', 'tel'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
lines.shape,linestest.shape

((2603, 2), (233, 2))

In [4]:
lines.eng=lines.eng.str.lower()
lines.tel=lines.tel.str.lower()
linestest.eng=linestest.eng.str.lower()#apply(lambda x: x.lower())
linestest.tel=linestest.tel.str.lower()

In [5]:
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', str(x)))
lines.tel=lines.tel.apply(lambda x: re.sub("'", '', str(x)))
linestest.eng=linestest.eng.apply(lambda x: re.sub("'", '', str(x)))
linestest.tel=linestest.tel.apply(lambda x: re.sub("'", '', str(x)))

In [6]:
exclude = set(string.punctuation)
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.tel=lines.tel.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
linestest.eng=linestest.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
linestest.tel=linestest.tel.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [7]:
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: re.sub("[0123456789]", "", x))
lines.tel = lines.tel.apply(lambda x: re.sub("[0123456789]", "", x))
linestest.eng=linestest.eng.apply(lambda x: re.sub("[0123456789]", "", x))
linestest.tel = linestest.tel.apply(lambda x: re.sub("[0123456789]", "", x))

In [8]:
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.tel=lines.tel.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.tel=lines.tel.apply(lambda x: re.sub(" +", " ", x))
lines.tel = lines.tel.apply(lambda x : 'S'+ x + 'E')
linestest.eng=linestest.eng.apply(lambda x: x.strip())
linestest.tel=linestest.tel.apply(lambda x: x.strip())
linestest.eng=linestest.eng.apply(lambda x: re.sub(" +", " ", x))
linestest.tel=linestest.tel.apply(lambda x: re.sub(" +", " ", x))

linestest.tel = linestest.tel.apply(lambda x : 'S'+ x + 'E')

In [9]:
all_eng_words=set()
for eng in lines.eng:
    for word in eng:
        if word not in all_eng_words:
            all_eng_words.add(word)
for eng in linestest.eng:
    for word in eng:
        if word not in all_eng_words:
            all_eng_words.add(word)

In [10]:
all_Telugu_words=set()
for tel in lines.tel:
    for word in tel:
        if word not in all_Telugu_words:
            all_Telugu_words.add(word)
for tel in linestest.tel:
    for word in tel:
        if word not in all_Telugu_words:
            all_Telugu_words.add(word)

In [11]:
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l))
for l in linestest.eng:
    lenght_list.append(len(l))
max_length_src = np.max(lenght_list)
print(max_length_src)

27


In [12]:
lenght_list=[]
for l in lines.tel:
    lenght_list.append(len(l))
for l in linestest.tel:
    lenght_list.append(len(l))
max_length_tar = np.max(lenght_list)

print(max_length_tar)

17


In [13]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_Telugu_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_Telugu_words)
print(num_encoder_tokens, num_decoder_tokens)
num_decoder_tokens += 1

61 64


In [14]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [15]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [16]:
lines = shuffle(lines)
print(lines.head(10))

                eng            tel
2238           విజయ         SవిజయE
1327          నన్నీ        Sనన్నుE
238       ఆనందించిన     SఆనందించుE
2157          వచ్చి        Sవచ్చుE
205          అసలుకి         SఅసలుE
1597         పెద్దల        Sపెద్దE
955        చెయ్యాలో       Sచెయ్యిE
783        గడిచింది        SగడుచుE
889   చిత్తశుద్ధితో  Sచిత్తశుద్ధిE
821       గుర్తించే    Sగుర్తించుE


In [17]:
X, y = lines.eng, lines.tel
X_test,y_test=linestest.eng,linestest.tel
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.001)
y_train.shape,y_test.shape,y_val.shape
X_test[3:4].values[0]

'అంత'

In [18]:
X_train.to_pickle('Weights_Tel/X_train.pkl')
X_test.to_pickle('Weights_Tel/X_test.pkl')

In [19]:
def generate_batch(X = X_train, y = y_train, batch_size = 10):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src+1),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar+1),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar+1, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text):
                    if t<len(target_text)-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [20]:
latent_dim = 100

In [21]:

# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens+1, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

Instructions for updating:
Colocations handled automatically by placer.


In [22]:
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [22]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [23]:
train_samples = len(X_train)
val_samples = len(X_val)
batch_size = 5
epochs = 100

train_samples,val_samples

(2600, 3)

In [ ]:
"""model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = int(train_samples/batch_size),validation_steps = int(val_samples/1),
                    epochs=epochs,
                    shuffle=True,validation_data = generate_batch(X_val, y_val, batch_size = 1))"""

In [ ]:
model.save_weights('nmt_weights_epoch100.h5')

In [24]:
model.load_weights('nmt_weights_epoch100.h5')

In [23]:
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [24]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    #print(target_token_index)
    target_seq[0, 0] = target_token_index['S']
   

    #print(target_seq)
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == 'E'  or len(decoded_sentence) > 100):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        #print(sampled_token_index)
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [25]:
def NN(NNinp):
    X_test[0]=NNinp
    test=[]
    test+=[X_test[0]]
    train_gen = generate_batch(test, [''], batch_size = 1)
    (input_seq, actual_output), _ = next(train_gen)
    decoded_sentence = decode_sequence(input_seq)
    inp=X_test[0:1].values[0]
    pred=decoded_sentence
    return pred[:-1]
    print('Input:', inp)
    print('Lemma:', pred[:-1])



In [26]:
def findPOS(word):
    mfinp=open("/home/anusha/Documents/sivareddyg-telugu-part-of-speech-tagger-9dbe2d21ca33/telugu.input.txt",'w',encoding='utf-8')
    mfop=open("/home/anusha/Documents/sivareddyg-telugu-part-of-speech-tagger-9dbe2d21ca33/telugu.output.txt",'r',encoding='utf-8')
    mfinp.write(word)
    mfinp.close()
    os.system('make tag -sC \'/home/anusha/Documents/sivareddyg-telugu-part-of-speech-tagger-9dbe2d21ca33\'')
    x=mfop.readline()
    a=x.split()
    #print(a)
    if  (('sg' in x) and (word[-2:]=='లు' or word[-1:]=='ల')):#(word[-3:]=='ులు' or word[-3:]=='ాలు' )) :
        return("sg")
    elif (('RB' in x) and (word[-2:]=='గా')):
        return("RB")
    elif(('sg' in x) and len(word)>4 and word[-4:]=='ాలలో'):

            return("sg")
    elif( word[-3:]=='ికి'):
        return(a[2].strip())
    elif(word[-1:]=='ై'):
        return(a[2].strip())

    else:
        return("not")

In [27]:
def rules(inp,maxw,mx,total,hstem):

    if(inp[-1]=='ం'):
        if(len(inp)>5 and inp[-5:]=='కోవటం' or inp[-5:]=='కోవడం'):
            maxw=inp[:-5]
            maxw+='కొను'
        elif(len(inp)>5 and inp[-5:]=='పోవడం' or inp[-5:]=='పోవటం'):
            #total+=1
            maxw=inp[:-5]
            maxw+='పోవు'
        elif(len(inp)>3 and inp[-3:]=='చడం' or inp[-3:]=='చటం'):
            maxw=inp[:-2]
            maxw+='ు'
        elif(len(inp)>len('చ్చాం') and inp[-5:]=='చ్చాం' ):
            maxw=inp[:-5]
            maxw+='చ్చు'
        else:
            maxw=inp

    elif(inp[-2:]=='లు'):
        if(inp[-3:]=='ులు' ):
            plural=findPOS(inp)

            if(plural=="not"):
                    maxw=inp[:-3]
                    maxw+='ి'
            else:
                maxw=inp

        elif(inp[-3:]=='ాలు'): #and mx!=1):
            temp=inp
            plural=findPOS(temp)
            if(plural=="not"):
                temp=temp[:-3]
                temp+='ం'
                maxw=temp
            else:
                maxw=temp
        else:
            plural=findPOS(inp)
            if(plural=="not"):
                if(len(inp)>len('ర్లు') and inp[-4:]=='ర్లు'):
                    maxw=inp[:-4]
                    maxw+='రు'
                    #print('ర్లు',inp,maxw)
                elif(len(inp)>len('న్లు') and inp[-4:]=='న్లు'):
                    maxw=inp[:-4]
                    maxw+='ను'
                    #print('న్లు',inp,maxw)
                elif(len(inp)>len('ట్లు') and inp[-4:]=='ట్లు'):
                    maxw=inp[:-4]
                    maxw+='ట్టు'
                    #print('ట్లు',inp,maxw)
                elif(len(inp)>len('ళ్లు') and inp[-4:]=='ళ్లు'):
                    maxw=inp[:-4]
                    maxw+='లు'
                    #print('ళ్లు',inp,maxw)
                elif(re.search('్లు$',inp)):
                    maxw=inp[:-3]
                    maxw+='ు'
                    #print('ళ్లు',inp,maxw)
                    

                else:
                    maxw=inp[:-2]
            else:
                maxw=inp
    elif(inp[-2:]=='చి'):
        maxw=inp[:-2]
        if(len(inp)>len('ిచి') and inp[-3:]=='ిచి'):
            maxw=inp[:-3]
            maxw+='ుచు'
        else:
            maxw+='చు'
    elif(len(inp)>=2 and inp[-2:]=='గా'):
        if(len(inp)==2):
            maxw=inp

        elif(len(inp)>len('లుగా') and inp[-4:]=='లుగా'):
            maxw,total=rules(inp[:-2],maxw,mx,total,hstem)

        elif(len(inp)>4 and inp[-4:]=='లాగా'):
            maxw=inp[:-4]
        elif(len(inp)>4 and inp[-4:]=='గ్గా'):
            maxw=inp[:-4]
            maxw+='కు'
        else:
            maxw=inp[:-2]
    elif(len(inp)>=2 and inp[-2:]=='లో'):
        if(inp==2):
            maxw=inp

        elif(len(inp)>len('ల్లో') and inp[-4:]=='ల్లో'):

            if(len(inp)>len('ళ్లల్లో') and inp[-7:]=='ళ్లల్లో' ):
                maxw=inp[:-4]
                maxw+='ు'
            elif(len(inp)>len('ళ్ళల్లో') and inp[-7:]=='ళ్ళల్లో' ):
                maxw=inp[:-4]
                maxw+='ు'
            elif(len(inp)>len('్లల్లో') and inp[-6:]=='్లల్లో'):
                maxw=inp[:-4]
                plural=findPOS(maxw)
                if(plural=="not"):
                    maxw,total=rules(maxw+'ు',maxw,mx,total,hstem)
                
            else:
                temp=inp[:-4]+"లు"
                maxw,total=rules(temp,maxw,mx,total,hstem)
        elif(len(inp)>len('లలో') and inp[-3:]=='లలో'):
            maxw=inp[:-2]
            plural=findPOS(maxw)
            if(plural=="not"):
                maxw,total=rules(maxw+'ు',maxw,mx,total,hstem)
        else:
            maxw=inp[:-2]
    elif(len(inp)>4 and inp[-4:]=='ల్ని'):
        if(len(inp)==4):
            maxw=inp
        elif(inp[-5:]=='ాల్ని'):
            maxw=inp[:-5]
            maxw+='ం'
        elif(inp[-5:]=='ుల్ని'):
            maxw=inp[:-5]
            maxw+='ి'
        else:
            maxw=inp[:-4]
    elif(inp[-2:]=='చే' or inp[-2:]=='చె'):
        if(len(inp)==2):
            maxw=inp
        elif(inp[-3:]=='ిచే' or inp[-3:]=='ిచె'):
            maxw=inp[:-3]
            maxw+='ుచు'
        else:
            maxw=inp[:-1]
            maxw+='ు'
    elif(inp[-2:]=='ను'):
        if(len(inp)==2):
            maxw=inp
        elif(len(inp)>6 and inp[-6:]=='స్తాను'):
            maxw=inp[:-6]
            maxw+='ంచు'
        elif(len(inp)>4 and inp[-4:]=='తాను'):
            if(inp[-5]=='డ'):
                maxw=inp[:-5]
                maxw+='ట్టు'
            else:
                maxw=inp[:-4]
        elif(len(inp)>4 and inp[-4:]=='యాను'):
            maxw=inp[:-4]
            maxw+='వు'
        elif(len(inp)>len('న్నాను') and inp[-6:]=='న్నాను'):
            maxw=inp[:-5]
            maxw+='ు'
        elif(len(inp)>len('రిగాను') and re.search('.*ి.ాను$',inp)):
            maxw=inp[:-3]
            maxw+='ు'
            maxw=list(maxw)
            maxw[-3]='ు'
            maxw=''.join(maxw)
        elif(len(inp)>3 and inp[-3:]=='ాను'):
            maxw=inp[:-3]
            maxw+='ు'
        elif(len(inp)>3 and inp[-3:]=='లను'):
            temp=inp[:-2]
            temp+='ు'
            maxw,total=rules(temp,maxw,mx,total,hstem)
        elif(len(inp)>len('న్ను') and inp[-4:]=='న్ను'):
            maxw=inp
        elif(len(inp)>4 and (inp[-4:]=='చేను' or inp[-4:]=='చెను')):
             maxw=inp[:-2]
             maxw,total=rules(maxw,maxw,mx,total,hstem)
        else:
            if(len(inp)>2):
                maxw=inp[:-2]
    elif(len(inp)>=2 and inp[-2:]=='డు'):
        if(len(inp)==2):
            maxw=inp
        elif(len(inp)>=3 and inp[-3:]=='ాడు'):

            if(len(inp)>len('డిగాడు') and re.search('.*ి.ాడు$',inp)):
                maxw=inp[:-3]
                maxw+='ు'
                maxw=list(maxw)
                maxw[-3]='ు'
                maxw=''.join(maxw)
            elif(len(inp)>len('న్నాడు') and inp[-6:]=='న్నాడు' ):
                maxw=inp[:-5]
                maxw+='ు'
            elif(len(inp)>len('తాడు') and inp[-4:]=='తాడు'):
                maxw=inp[:-4]
            elif(len(inp)>len('యాడు') and inp[-4:]=='యాడు'):
                maxw=inp[:-4]
                maxw+='వు'
            elif(len(inp)>len('శాడు') and (inp[-4:]=='శాడు') or inp[-4:]=='సాడు'):
                maxw=inp[:-4]
                maxw+='యు'
            elif(len(inp)>len('డ్డాడు') and inp[-6:]=='డ్డాడు'):
                maxw=inp[:-6]
                maxw+='డు'
            else:
                maxw=inp[:-3]
                maxw+='ు'
        elif(len(inp)>len('ిలేడు') and re.search('.+ి.ేడు$',inp)):
            maxw=inp[:-3]
            maxw+='ు'
            maxw=list(maxw)
            maxw[-3]='ు'
            maxw=''.join(maxw)
        elif(len(inp)>len('ేడు') and inp[-3:]=='ేడు'):
            maxw=inp[:-3]
            maxw+='ు'
        else:
            maxw=inp
    elif(len(inp)>=2 and inp[-2:]=='ని'):
        if(len(inp)==2):
            maxw=inp
        elif(inp[-4:]=='కుని'):
            maxw=inp[:-4]
            maxw+='కొను'
        elif(len(inp)>len('ాన్ని') and inp[-5:]=='ాన్ని'):
            maxw=inp[:-5]
            maxw+='ం'
        elif(len(inp)>len('న్ని') and inp[-4:]=='న్ని'):
            maxw=inp
        elif(len(inp)>len('్ని') and inp[-3:]=='్ని'):
            maxw=inp[:-3]
            maxw+='ు'
        elif(len(inp)>len('కొని') and inp[-4:]=='కొని'):
            if(len(inp)>len('క్కొని') and inp[-6:]=='క్కొని'):
                maxw=inp[:-6]
                maxw+='గు'
            else:
                maxw=inp[:-4]
        elif(len(inp)>len('లోని') and inp[-4:]=='లోని'):
            maxw,total=rules(inp[:-2],maxw,mx,total,hstem)
        elif(len(inp)>3 and inp[-3:]=='లని'):
            maxw=inp[:-2]
            maxw+='ు'
            maxw,total=rules(maxw,maxw,mx,total,hstem)
        else:
            maxw=inp[:-2]
    elif(len(inp)>=2 and inp[-2:]=='తో'):
        if(len(inp)==2):
            maxw=inp
        if(len(inp)>4 and inp[-4:]=='ాలతో'):
            temp=inp[:-2]
            temp+='ు'
            plural=findPOS(temp)
            if(plural=="not"):
                maxw=temp[:-3]
                maxw+='ం'
        elif(len(inp)>3 and inp[-3:]=='లతో'):
            maxw=inp[:-2]
            maxw=maxw+'ు'
            plural=findPOS(maxw)
            if(plural=="not"):
                maxw,total=rules(maxw,maxw,mx,total,hstem)
        elif(len(inp)>len('ల్తో') and inp[-4:]=='ల్తో'):
            maxw=inp[:-3]
            maxw+='ు'
            plural=findPOS(maxw)
            if(plural=="not"):
                maxw,total=rules(maxw,maxw,mx,total,hstem)
        elif(len(inp)>len('త్తో') and inp[-4:]=='త్తో'):
            maxw=inp[:-3]
            maxw+='ి'
        else:
            maxw=inp[:-2]
    elif(inp[-1]=='ల'):

        maxw=inp

        plural=findPOS(maxw)
        if(plural=="not"):
            maxw,total=rules(maxw+'ు',maxw,mx,total,hstem)
    elif(len(inp)>=2 and inp[-2:]=='కు'):
        if(len(inp)==2):
            maxw=inp
        elif(len(inp)>3 and inp[-3:]=='లకు'):
            plural=findPOS(inp[:-2])
            if(plural=='not'):
                temp=inp[:-2]
                maxw,total=rules(temp+'ు',maxw,mx,total,hstem)
            else:
                maxw=inp[:-2]
        elif(len(inp)>3 and inp[-3:]=='నకు'):
            maxw=inp[:-3]
        else:
            maxw=inp[:-2]
    elif(len(inp)>=3 and inp[-3:]=='ంది'):

        if(len(inp)>len('ింది') and inp[-4:]=='ింది'):
            if(len(inp)>len('ిగింది') and re.search('.+ి.ింది$',inp)):
                maxw=inp[:-4]
                maxw+='ు'
                maxw=list(maxw)
                maxw[-3]='ు'
                maxw=''.join(maxw)
            elif(len(inp)>len('ేసింది') and inp[-6:]=='ేసింది'):
                maxw=inp[:-6]
                maxw+='ు'
            elif(len(inp)>len('యింది') and inp[-5:]=='యింది'):
                maxw=inp[:-5]
                maxw+='వు'
            else:
                maxw=inp[:-4]
                maxw+='ు'
        elif(len(inp)>len('స్తుంది') and inp[-7:]=='స్తుంది'):
            maxw=inp[:-7]
            maxw+='యు'
        elif(len(inp)>len('తుంది') and inp[-5:]=='తుంది'):
            maxw=inp[:-5]
            maxw+='వు'
        elif(len(inp)>len('కుంది') and inp[-5:]=='కుంది'):
            maxw=inp[:-5]
            maxw+='కొను'
        elif(len(inp)>len('ొంది') and inp[-4:]=='ొంది'):
            maxw=inp[:-3]
            maxw+='ను'

        else:
            maxw=inp
    elif(len(inp)>len('న') and inp[-1:]=='న' and inp[-3:]!='న్న' ):

        if(len(inp)>len('ిలిన') and re.search('.+ి.ిన$',inp)):

            maxw=inp[:-2]
            maxw+='ు'
            maxw=list(maxw)
            maxw[-3]='ు'
            maxw=''.join(maxw)
        elif(len(inp)>len('ాల్సిన') and re.search('.+ాల్సిన$',inp)):
            maxw=inp[:-6]
            maxw+='ు'
        elif(len(inp)>len('సిన') and inp[-3:]=='సిన'):

            maxw=inp[:-3]
            maxw+='యు'
        elif(len(inp)>len('యిన') and inp[-3:]=='యిన'):

            maxw=inp[:-3]
            maxw+='వు'
        elif(len(inp)>2 and inp[-2:]=='ిన'):

            maxw=inp[:-2]
            maxw+='ు'
        elif(len(inp)>len('ైన') and inp[-2:]=='ైన'):
            maxw=inp[:-2]
            maxw+='ు'
        elif(len(inp)>len('ున') and inp[-2:]=='ున'):
            maxw=inp[:-1]
        else:
            maxw=inp
    elif(len(inp)>2 and (inp[-2:]=='కి' or inp[-2:]=="కీ")):

        if(len(inp)>3 and (inp[-3:]=='లకి'or inp[-3:]=='లకీ')):
            maxw=inp[:-2]
            plural=findPOS(maxw)
            if(plural=="not"):
                maxw,total=rules(maxw+'ు',maxw,mx,total,hstem)
        elif(len(inp)>len('ానికి') and (inp[-5:]=='ానికి' or inp[-5:]=='ానికీ')):
            maxw=inp[:-5]
            maxw+='ం'
            maxw,total=rules(maxw,maxw,mx,total,hstem)

        elif(len(inp)>len('ికి') and (inp[-3:]=='ికి' or inp[-3:]=='ికీ')):
            pos=findPOS(inp)
            if(pos=="NN" or pos=="NNP"):
                maxw=inp[:-2]
            elif(pos=="NST"):
                maxw=inp[:-3]
            else:
                maxw=inp[:-3]
                maxw+='ు'
        elif(len(inp)>len('లోకి') and (inp[-4:]=='లోకి' or inp[-4:]=='లోకీ')):
            maxw=inp[:-2]
            maxw,total=rules(maxw,maxw,mx,total,hstem)
        else:
            maxw=inp[:-2]
    elif(len(inp)>4 and inp[-4:]=='స్తూ'):

        if(inp[-5:]=='ుస్తూ'):
            maxw=inp[:-4]
            maxw+='పు'
        elif(inp[-5:]=='ూస్తూ'):
            maxw=inp[:-4]
            maxw+='డు'
        elif(inp[-5:]=='ిస్తూ'):
            maxw=inp[:-4]
            maxw+='ంచు'
        else:
            maxw=inp[:-4]
            maxw+='యు'
    elif(len(inp)>1 and inp[-1:]=='ై'):
        if(len(inp)>2 and inp[-3:]=='లపై'):
            maxw=inp[:-2]
            maxw,total=rules(maxw+'ు',maxw,mx,total,hstem)
        elif(len(inp)>=2 and inp[-2:]=='పై'):
            if(len(inp)==2):
                maxw=inp

            else:
                maxw=inp[:-2]
        else:
            pos=findPOS(inp)
            if(pos!='QC'):
                maxw=inp[:-1]
                maxw+='ు'
            else:
                maxw=inp
        if (not ((hstem[inp]==maxw) or ((hstem[inp][:-1]==maxw[:-2]) and (hstem[inp][-1]=='ం' and maxw[-2:]=='ము')) or ((hstem[inp][:-2]==maxw[:-1]) and (maxw[-1]=='ం' and hstem[inp][-2:]=='ము')))):
                total+=1
                """print(total,inp,hstem[inp],maxw)"""
    elif(inp[-1]=='ట' or inp[-1]=='త'):
        maxw=inp
    else:

        maxw=NN(inp)
        total=0
    return maxw,total

            
        



In [31]:

inp='పాపాలు'
print("input :"+inp)
maxw,_=rules(inp,'',1,1,{})
print("Lemma :"+maxw)

input :పాపాలు
Lemma :పాపం
